In [3]:
import pandas as pd
import pyodbc

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

In [4]:
connection = pyodbc.connect(
  r'Driver={ODBC Driver 17 for SQL Server};'
  r'Server=localhost;'
  r'Database=AdventureWorks2022;'
  r'Trusted_Connection=yes;'
)

In [5]:
#class for reading table
class db_reader:
    def read_table(self, table_name, connection, col=None):
        if col:
            query = f"SELECT {', '.join(col)} FROM {table_name}" #', '.join(col) converts the list of columns into a comma-separated string. For example, if col = ['name', 'salary'], ', '.join(col) produces 'name, salary'.
        else:
            query = f"SELECT * FROM {table_name}"
        return pd.read_sql_query(query, connection)

In [6]:
#class for merging
class merge:
  def merge_tbl(self, t1, t2, join, col = None, col1 = None, col2 = None):
    if col1 and col2:
      return pd.merge(t1, t2, how = join, left_on = col1, right_on = col2)
    elif col:
      return pd.merge(t1, t2, how = join, on = col)

In [7]:
# 301 Get the list of customer IDs and names for customers who have not placed any orders

cus = db_reader().read_table('sales.customer', connection)
cus.head()

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
0,2,NaN,1028.0,1,AW00000002,E552F657-A9AF-4A7D-A645-C429D6E02491,2014-09-12 11:15:07.263
1,3,NaN,642.0,4,AW00000003,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2014-09-12 11:15:07.263
2,4,NaN,932.0,4,AW00000004,FF862851-1DAA-4044-BE7C-3E85583C054D,2014-09-12 11:15:07.263
3,5,NaN,1026.0,4,AW00000005,83905BDC-6F5E-4F71-B162-C98DA069F38A,2014-09-12 11:15:07.263
4,6,NaN,644.0,4,AW00000006,1A92DF88-BFA2-467D-BD54-FCB9E647FDD7,2014-09-12 11:15:07.263


In [8]:
per = db_reader().read_table('person.person', connection)
per.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30


In [9]:
name = merge().merge_tbl(cus[['PersonID']], per[['BusinessEntityID', 'FirstName', 'LastName']], 'outer', col1 = 'PersonID',col2 = 'BusinessEntityID')
name.head()

,PersonID,BusinessEntityID,FirstName,LastName
0,NaN,1.0,Ken,Sánchez
1,NaN,2.0,Terri,Duffy
2,NaN,3.0,Roberto,Tamburello
3,NaN,4.0,Rob,Walters
4,NaN,5.0,Gail,Erickson


In [10]:
soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [11]:
name[~name['PersonID'].isin(soh['CustomerID'])].head()

,PersonID,BusinessEntityID,FirstName,LastName
0,NaN,1.0,Ken,Sánchez
1,NaN,2.0,Terri,Duffy
2,NaN,3.0,Roberto,Tamburello
3,NaN,4.0,Rob,Walters
4,NaN,5.0,Gail,Erickson


In [12]:
# 302 Get the list of orders that do not contain products with IDs 16, 21, and 26

sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [13]:
sod[~sod['ProductID'].isin([16,21,26])].head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [14]:
# 303 Retrieve the product names for products not belonging to categories with IDs 2, 5, and 8

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [15]:
psc = db_reader().read_table('production.productsubcategory', connection)
psc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [16]:
c = merge().merge_tbl(p[['ProductID', 'Name', 'ProductSubcategoryID']], psc[['ProductSubcategoryID', 'ProductCategoryID']], 'inner', 'ProductSubcategoryID')
c.head()

,ProductID,Name,ProductSubcategoryID,ProductCategoryID
0,680,"HL Road Frame - Black, 58",14.0,2
1,706,"HL Road Frame - Red, 58",14.0,2
2,707,"Sport-100 Helmet, Red",31.0,4
3,708,"Sport-100 Helmet, Black",31.0,4
4,709,"Mountain Bike Socks, M",23.0,3


In [17]:
c[~c['ProductCategoryID'].isin([2,5,8])].head()

,ProductID,Name,ProductSubcategoryID,ProductCategoryID
2,707,"Sport-100 Helmet, Red",31.0,4
3,708,"Sport-100 Helmet, Black",31.0,4
4,709,"Mountain Bike Socks, M",23.0,3
5,710,"Mountain Bike Socks, L",23.0,3
6,711,"Sport-100 Helmet, Blue",31.0,4


In [18]:
# 304 Retrieve the product names and list prices for products with a list price less than $50 or a color of 'Blue'

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [19]:
p[(p['ListPrice'] < 50) & (p['Color'] == 'Blue')].head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
215,711,"Sport-100 Helmet, Blue",HL-U509-B,False,True,Blue,4,3,13.0863,34.99,None,None,None,NaN,0,S,None,None,31.0,33.0,2011-05-31,NaT,NaT,FD7C0858-4179-48C2-865B-ABD5DFC7BC1D,2014-02-08 10:01:36.827


In [20]:
# 305 Get the list of employees hired after '2005-01-01' and belonging to the department with ID 5

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']

In [21]:
emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [22]:
d = db_reader().read_table('humanresources.employeedepartmenthistory', connection)
d.head()

,BusinessEntityID,DepartmentID,ShiftID,StartDate,EndDate,ModifiedDate
0,1,16,1,2009-01-14,None,2009-01-13
1,2,1,1,2008-01-31,None,2008-01-30
2,3,1,1,2007-11-11,None,2007-11-10
3,4,1,1,2007-12-05,2010-05-30,2010-05-28
4,4,2,1,2010-05-31,None,2010-05-30


In [23]:
de = merge().merge_tbl(emp[['BusinessEntityID', 'HireDate']], d[['BusinessEntityID', 'DepartmentID']], 'inner', 'BusinessEntityID')
de.head()

,BusinessEntityID,HireDate,DepartmentID
0,1,2009-01-14,16
1,2,2008-01-31,1
2,3,2007-11-11,1
3,4,2007-12-05,1
4,4,2007-12-05,2


In [24]:
de[(pd.to_datetime(de['HireDate']) > '2005-01-01') & (de['DepartmentID'] ==5)].head()

,BusinessEntityID,HireDate,DepartmentID
16,16,2007-12-20,5
255,250,2011-02-25,5
256,251,2009-02-10,5
257,252,2009-02-28,5
258,253,2009-12-17,5


In [25]:
# 306 Get the list of orders placed between '2010-01-01' and '2010-12-31'

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [26]:
soh[(pd.to_datetime(soh['OrderDate']) > '2010-01-01') & (pd.to_datetime(soh['OrderDate']) < '2010-12-31')].head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate


In [27]:
# 307 Retrieve the product names with a list price between $50 and $100

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [28]:
p[(p['ListPrice'] > 50) & (p['ListPrice'] < 100)].head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
313,809,ML Mountain Handlebars,HB-M763,True,True,None,500,375,27.4925,61.920,None,None,None,NaN,1,M,M,None,4.0,54.0,2012-05-30,NaT,NaT,AE6020DF-D9C9-4D34-9795-1F80E6BBF5A5,2014-02-08 10:01:36.827
316,812,ML Road Handlebars,HB-R720,True,True,None,500,375,27.4925,61.920,None,None,None,NaN,1,R,M,None,4.0,57.0,2012-05-30,NaT,NaT,33FA1A03-38D6-405E-BE9B-EEA92F3D204F,2014-02-08 10:01:36.827
319,815,LL Mountain Front Wheel,FW-M423,True,True,Black,500,375,26.9708,60.745,None,None,None,NaN,1,M,L,None,17.0,42.0,2012-05-30,2013-05-29,NaT,D7B1D161-182E-44F6-A9FF-9C1EBA76613B,2014-02-08 10:01:36.827
322,818,LL Road Front Wheel,FW-R623,True,True,Black,500,375,37.9909,85.565,None,None,G,900.0,1,R,L,None,17.0,49.0,2012-05-30,2013-05-29,NaT,9E66DE78-DECB-438A-B9A9-023C773C60A2,2014-02-08 10:01:36.827
327,823,LL Mountain Rear Wheel,RW-M423,True,True,Black,500,375,38.9588,87.745,None,None,None,NaN,1,M,L,None,17.0,123.0,2012-05-30,2013-05-29,NaT,E6C39F58-995F-4786-A309-DF3812D8B30F,2014-02-08 10:01:36.827


In [29]:
# 308 Retrieve the customer IDs and names for customers with a total amount spent greater than $1000

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [30]:
cus = db_reader().read_table('sales.customer', connection)
cus.head()

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
0,2,NaN,1028.0,1,AW00000002,E552F657-A9AF-4A7D-A645-C429D6E02491,2014-09-12 11:15:07.263
1,3,NaN,642.0,4,AW00000003,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2014-09-12 11:15:07.263
2,4,NaN,932.0,4,AW00000004,FF862851-1DAA-4044-BE7C-3E85583C054D,2014-09-12 11:15:07.263
3,5,NaN,1026.0,4,AW00000005,83905BDC-6F5E-4F71-B162-C98DA069F38A,2014-09-12 11:15:07.263
4,6,NaN,644.0,4,AW00000006,1A92DF88-BFA2-467D-BD54-FCB9E647FDD7,2014-09-12 11:15:07.263


In [31]:
name = db_reader().read_table('person.person', connection)
name.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30


In [32]:
c_name = merge().merge_tbl(cus['PersonID'], name[['BusinessEntityID', 'FirstName', 'LastName']], 'inner', col1 = 'PersonID', col2 = 'BusinessEntityID')
c_name.head()

,PersonID,BusinessEntityID,FirstName,LastName
0,13531.0,13531,Jon,Yang
1,5454.0,5454,Eugene,Huang
2,11269.0,11269,Ruben,Torres
3,11358.0,11358,Christy,Zhu
4,11901.0,11901,Elizabeth,Johnson


In [33]:
sale = merge().merge_tbl(soh[['CustomerID', 'TotalDue']], c_name[['BusinessEntityID', 'FirstName', 'LastName']], 'inner', col1 = 'CustomerID', col2 = 'BusinessEntityID')
sale.head()

,CustomerID,TotalDue,BusinessEntityID,FirstName,LastName
0,14501,772.5036,14501,Bryan,Gray
1,11003,3756.9890,11003,Jenna,Young
2,16624,3953.9884,16624,Nichole,Goel
3,11005,3729.3640,11005,Joe,Madan
4,11011,3756.9890,11011,Joe,Rana


In [34]:
sale[sale['TotalDue'] > 1000].head()

,CustomerID,TotalDue,BusinessEntityID,FirstName,LastName
1,11003,3756.9890,11003,Jenna,Young
2,16624,3953.9884,16624,Nichole,Goel
3,11005,3729.3640,11005,Joe,Madan
4,11011,3756.9890,11011,Joe,Rana
6,16351,3953.9884,16351,Luke,Nelson


In [35]:
# 309 Get the list of products with a weight greater than 10 pounds

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [36]:
p[p['Weight'] > 10].head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
179,507,LL Mountain Rim,RM-M464,False,False,None,800,600,0.0,0.0,None,None,G,435.0,0,None,L,None,NaN,NaN,2008-04-30,NaT,NaT,B2CC7DFB-783D-4587-88C0-2712A538A5B2,2014-02-08 10:01:36.827
180,508,ML Mountain Rim,RM-M692,False,False,None,800,600,0.0,0.0,None,None,G,450.0,0,None,M,None,NaN,NaN,2008-04-30,NaT,NaT,733FD04D-322F-44F5-BEEC-F326189D1CE6,2014-02-08 10:01:36.827
181,509,HL Mountain Rim,RM-M823,False,False,None,800,600,0.0,0.0,None,None,G,400.0,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9FA4A3B5-D396-48D4-ADFC-B573BC4A800A,2014-02-08 10:01:36.827
182,510,LL Road Rim,RM-R436,False,False,None,800,600,0.0,0.0,None,None,G,445.0,0,None,L,None,NaN,NaN,2008-04-30,NaT,NaT,C2770757-B258-4EEC-A811-6856FAF87437,2014-02-08 10:01:36.827
183,511,ML Road Rim,RM-R600,False,False,None,800,600,0.0,0.0,None,None,G,450.0,0,None,M,None,NaN,NaN,2008-04-30,NaT,NaT,80108059-0002-4253-A805-53A2324C33A4,2014-02-08 10:01:36.827


In [37]:
# 310 Write a query to retrieve the employee names along with a new column "EmployeeType" that categorizes employees as "Full-Time" if their job title contains 'Manager', and "Part-Time" otherwise

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']

In [38]:
emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [39]:
emp['EmployeeType'] = None

In [40]:
emp.loc[emp['JobTitle'].str.contains('Manager'), 'EmployeeType'] = 'Full-Time'

In [41]:
emp.loc[emp['EmployeeType'].isna(), 'EmployeeType'] = 'Part-Time'

In [42]:
emp[['BusinessEntityID', 'EmployeeType']].head()

,BusinessEntityID,EmployeeType
0,1,Part-Time
1,2,Part-Time
2,3,Full-Time
3,4,Part-Time
4,5,Part-Time


In [43]:
# 311 Retrieve the product names along with a new column "DiscountType" that categorizes products as "Discounted" if their standard cost is less than the list price, and "Regular Price" otherwise

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [44]:
p['DiscountType'] = None

In [45]:
p.loc[(p['StandardCost']) < (p['ListPrice']), 'DiscountType'] = 'Discounted'

In [46]:
p.loc[p['DiscountType'].isna(), 'DiscountType'] = 'Regular Price'

In [47]:
p[['Name', 'DiscountType']].head()

,Name,DiscountType
0,Adjustable Race,Discounted
1,Bearing Ball,Regular Price
2,BB Ball Bearing,Regular Price
3,Headset Ball Bearings,Regular Price
4,Blade,Regular Price


In [48]:
# 312 Get the list of product names and list prices for products with IDs 3, 7, and 12

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [49]:
a = p[p['ProductID'].isin([3,7,12])]

In [50]:
a[['ProductID', 'Name', 'ListPrice']]

,ProductID,Name,ListPrice
2,3,BB Ball Bearing,0.0


In [51]:
# 313 Retrieve the order IDs and order dates for orders with IDs 1002, 1006, and 1011

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [52]:
soh[soh['SalesOrderID'].isin([1002,1006,1011])]

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate


In [53]:
# 314 Get the list of employee names and titles for employees with titles containing the word 'Assistant'

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']
emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [54]:
a = emp[emp['JobTitle'].str.contains('Assistant')]
a.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
16,17,253022876,adventure-works\kevin0,2.0,Marketing Assistant,1987-05-03,S,M,2007-01-26,False,42,41,True,1B480240-95C0-410F-A717-EB29943C8886,2014-06-30
18,19,52541318,adventure-works\mary2,2.0,Marketing Assistant,1978-01-29,S,F,2011-02-14,False,43,41,True,1F6DA901-C7F7-48A8-8EEF-D81868D72B52,2014-06-30
19,20,323403273,adventure-works\wanida0,2.0,Marketing Assistant,1975-03-17,M,F,2011-01-07,False,41,40,True,43CCA446-DA1C-454C-8530-873AD2923E1B,2014-06-30
218,219,242381745,adventure-works\sean1,4.0,Document Control Assistant,1987-03-12,S,M,2009-01-22,False,78,59,True,F78657A9-83EF-4B4D-9809-3C4C6D1A7BB4,2014-06-30
219,220,260770918,adventure-works\karen0,4.0,Document Control Assistant,1975-12-25,M,F,2009-02-09,False,79,59,True,8A41ED23-7AE2-4C15-AA5D-9496E721F848,2014-06-30


In [55]:
per = db_reader().read_table('person.person', connection)
per.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30


In [56]:
res = merge().merge_tbl(a[['BusinessEntityID', 'JobTitle']], per[['BusinessEntityID', 'FirstName', 'LastName']], 'inner', 'BusinessEntityID')
res.head()

,BusinessEntityID,JobTitle,FirstName,LastName
0,17,Marketing Assistant,Kevin,Brown
1,19,Marketing Assistant,Mary,Dempsey
2,20,Marketing Assistant,Wanida,Benshoof
3,219,Document Control Assistant,Sean,Chai
4,220,Document Control Assistant,Karen,Berge


In [57]:
# 315 Retrieve the list of product names for products with names containing the word 'helmet'

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [58]:
p[p['Name'].str.contains('helmet' ,case= False)]

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
211,707,"Sport-100 Helmet, Red",HL-U509-R,False,True,Red,4,3,13.0863,34.99,None,None,None,NaN,0,S,None,None,31.0,33.0,2011-05-31,NaT,NaT,2E1EF41A-C08A-4FF6-8ADA-BDE58B64A712,2014-02-08 10:01:36.827
212,708,"Sport-100 Helmet, Black",HL-U509,False,True,Black,4,3,13.0863,34.99,None,None,None,NaN,0,S,None,None,31.0,33.0,2011-05-31,NaT,NaT,A25A44FB-C2DE-4268-958F-110B8D7621E2,2014-02-08 10:01:36.827
215,711,"Sport-100 Helmet, Blue",HL-U509-B,False,True,Blue,4,3,13.0863,34.99,None,None,None,NaN,0,S,None,None,31.0,33.0,2011-05-31,NaT,NaT,FD7C0858-4179-48C2-865B-ABD5DFC7BC1D,2014-02-08 10:01:36.827


In [59]:
# 316 Retrieve the product names that have no associated product manuals


In [60]:
# 317 Get the list of customer IDs and names for customers who have not placed any orders in the year 2020

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [61]:
cus = db_reader().read_table('person.person', connection)
cus.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30


In [62]:
name = merge().merge_tbl(soh[['CustomerID', 'OrderDate']], cus[['BusinessEntityID', 'FirstName', 'LastName']], 'inner', col1 = 'CustomerID', col2 = 'BusinessEntityID')
name.head()

,CustomerID,OrderDate,BusinessEntityID,FirstName,LastName
0,14501,2011-05-31,14501,Bryan,Gray
1,11003,2011-05-31,11003,Jenna,Young
2,16624,2011-06-01,16624,Nichole,Goel
3,11005,2011-06-01,11005,Joe,Madan
4,11011,2011-06-01,11011,Joe,Rana


In [63]:
name.loc[name['OrderDate'].dt.year == 2020]

,CustomerID,OrderDate,BusinessEntityID,FirstName,LastName


In [64]:
# 318 Get the list of orders that do not contain products with IDs 17, 22, and 27

sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()


,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [65]:
sod[sod['ProductID'].isin([17,22,27])]

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate


In [66]:
# 319 Retrieve the product names for products not belonging to categories with IDs 3, 6, and 9

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [67]:
psc = db_reader().read_table('production.productsubcategory', connection)
psc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [68]:
c = merge().merge_tbl(p[['ProductID', 'Name', 'ProductSubcategoryID']], psc[['ProductSubcategoryID', 'ProductCategoryID']], 'inner', 'ProductSubcategoryID')
c.head()

,ProductID,Name,ProductSubcategoryID,ProductCategoryID
0,680,"HL Road Frame - Black, 58",14.0,2
1,706,"HL Road Frame - Red, 58",14.0,2
2,707,"Sport-100 Helmet, Red",31.0,4
3,708,"Sport-100 Helmet, Black",31.0,4
4,709,"Mountain Bike Socks, M",23.0,3


In [69]:
c[~c['ProductCategoryID'].isin([3,6,9])].head()

,ProductID,Name,ProductSubcategoryID,ProductCategoryID
0,680,"HL Road Frame - Black, 58",14.0,2
1,706,"HL Road Frame - Red, 58",14.0,2
2,707,"Sport-100 Helmet, Red",31.0,4
3,708,"Sport-100 Helmet, Black",31.0,4
6,711,"Sport-100 Helmet, Blue",31.0,4


In [70]:
# 320 Retrieve the product names and list prices for products with a list price less than $50 and a color of 'Black'

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [71]:
p[(p['ListPrice'] < 50) & (p['Color'] == 'Black')].head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
5,317,LL Crankarm,CA-5965,False,False,Black,500,375,0.0000,0.00,None,None,None,NaN,0,None,L,None,NaN,NaN,2008-04-30,NaT,NaT,3C9D10B7-A6B2-4774-9963-C19DCEE72FEA,2014-02-08 10:01:36.827
6,318,ML Crankarm,CA-6738,False,False,Black,500,375,0.0000,0.00,None,None,None,NaN,0,None,M,None,NaN,NaN,2008-04-30,NaT,NaT,EABB9A92-FA07-4EAB-8955-F0517B4A4CA7,2014-02-08 10:01:36.827
7,319,HL Crankarm,CA-7457,False,False,Black,500,375,0.0000,0.00,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,7D3FD384-4F29-484B-86FA-4206E276FE58,2014-02-08 10:01:36.827
10,322,Chainring,CR-7833,False,False,Black,1000,750,0.0000,0.00,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,F0AC2C4D-1A1F-4E3C-B4D9-68AEA0EC1CE4,2014-02-08 10:01:36.827
212,708,"Sport-100 Helmet, Black",HL-U509,False,True,Black,4,3,13.0863,34.99,None,None,None,NaN,0,S,None,None,31.0,33.0,2011-05-31,NaT,NaT,A25A44FB-C2DE-4268-958F-110B8D7621E2,2014-02-08 10:01:36.827


In [72]:
# 321 Get the list of employees hired after '2008-01-01' and belonging to either the department with ID 5 or the department with ID 8

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']
emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [73]:
d = db_reader().read_table('humanresources.employeedepartmenthistory', connection)
d.head()

,BusinessEntityID,DepartmentID,ShiftID,StartDate,EndDate,ModifiedDate
0,1,16,1,2009-01-14,None,2009-01-13
1,2,1,1,2008-01-31,None,2008-01-30
2,3,1,1,2007-11-11,None,2007-11-10
3,4,1,1,2007-12-05,2010-05-30,2010-05-28
4,4,2,1,2010-05-31,None,2010-05-30


In [74]:
de = merge().merge_tbl(emp[['BusinessEntityID', 'HireDate']], d[['BusinessEntityID', 'DepartmentID']], 'inner', 'BusinessEntityID')
de.head()

,BusinessEntityID,HireDate,DepartmentID
0,1,2009-01-14,16
1,2,2008-01-31,1
2,3,2007-11-11,1
3,4,2007-12-05,1
4,4,2007-12-05,2


In [75]:
de[(pd.to_datetime(de['HireDate']) > '2008-01-01') & (de['DepartmentID'] ==5) | (de['DepartmentID'] ==8)].head()

,BusinessEntityID,HireDate,DepartmentID
27,26,2008-12-01,8
223,222,2008-12-12,8
224,223,2009-01-26,8
226,224,2009-01-07,8
227,225,2009-02-13,8


In [76]:
# 322 Get the list of orders placed between '2015-01-01' and '2015-12-31'

soh = db_reader().read_table('sales.salesorderheader', connection)

In [77]:
soh[(pd.to_datetime(soh['OrderDate']) > '2015-01-01') & (pd.to_datetime(soh['OrderDate']) < '2015-12-31')].head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate


In [78]:
# 323 Retrieve the product names with a weight between 5 and 10 pounds

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [79]:
p[(p['Weight'] > 5) & (p['Weight'] < 10)]

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate


In [80]:
# 324 Retrieve the customer IDs and names for customers with a total amount spent greater than $5000

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [81]:
cus = db_reader().read_table('sales.customer', connection)
cus.head()

,CustomerID,PersonID,StoreID,TerritoryID,AccountNumber,rowguid,ModifiedDate
0,2,NaN,1028.0,1,AW00000002,E552F657-A9AF-4A7D-A645-C429D6E02491,2014-09-12 11:15:07.263
1,3,NaN,642.0,4,AW00000003,130774B1-DB21-4EF3-98C8-C104BCD6ED6D,2014-09-12 11:15:07.263
2,4,NaN,932.0,4,AW00000004,FF862851-1DAA-4044-BE7C-3E85583C054D,2014-09-12 11:15:07.263
3,5,NaN,1026.0,4,AW00000005,83905BDC-6F5E-4F71-B162-C98DA069F38A,2014-09-12 11:15:07.263
4,6,NaN,644.0,4,AW00000006,1A92DF88-BFA2-467D-BD54-FCB9E647FDD7,2014-09-12 11:15:07.263


In [82]:
name = db_reader().read_table('person.person', connection)
name.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30


In [83]:
c_name = merge().merge_tbl(cus['PersonID'], name[['BusinessEntityID', 'FirstName', 'LastName']], 'inner', col1 = 'PersonID', col2 = 'BusinessEntityID')
c_name.head()

,PersonID,BusinessEntityID,FirstName,LastName
0,13531.0,13531,Jon,Yang
1,5454.0,5454,Eugene,Huang
2,11269.0,11269,Ruben,Torres
3,11358.0,11358,Christy,Zhu
4,11901.0,11901,Elizabeth,Johnson


In [84]:
sale = merge().merge_tbl(soh[['CustomerID', 'TotalDue']], c_name[['BusinessEntityID', 'FirstName', 'LastName']], 'inner', col1 = 'CustomerID', col2 = 'BusinessEntityID')
sale.head()

,CustomerID,TotalDue,BusinessEntityID,FirstName,LastName
0,14501,772.5036,14501,Bryan,Gray
1,11003,3756.9890,11003,Jenna,Young
2,16624,3953.9884,16624,Nichole,Goel
3,11005,3729.3640,11005,Joe,Madan
4,11011,3756.9890,11011,Joe,Rana


In [85]:
a = sale.groupby(['CustomerID', 'FirstName', 'LastName'])['TotalDue'].sum().reset_index().head()

In [86]:
a[a['TotalDue'] > 5000].head()

,CustomerID,FirstName,LastName,TotalDue
0,11000,Mary,Young,9115.1341
1,11001,Amber,Young,7054.1875
2,11002,Courtney,Young,8966.0143
3,11003,Jenna,Young,8993.9155
4,11004,Chloe,Harris,9056.5911


In [87]:
# 325 Get the list of products with a list price greater than twice their standard cost

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [88]:
p[p['ListPrice'] > 2*p['StandardCost']].head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0000,1000.00,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
211,707,"Sport-100 Helmet, Red",HL-U509-R,False,True,Red,4,3,13.0863,34.99,None,None,None,NaN,0,S,None,None,31.0,33.0,2011-05-31,NaT,NaT,2E1EF41A-C08A-4FF6-8ADA-BDE58B64A712,2014-02-08 10:01:36.827
212,708,"Sport-100 Helmet, Black",HL-U509,False,True,Black,4,3,13.0863,34.99,None,None,None,NaN,0,S,None,None,31.0,33.0,2011-05-31,NaT,NaT,A25A44FB-C2DE-4268-958F-110B8D7621E2,2014-02-08 10:01:36.827
213,709,"Mountain Bike Socks, M",SO-B909-M,False,True,White,4,3,3.3963,9.50,M,None,None,NaN,0,M,None,U,23.0,18.0,2011-05-31,2012-05-29,NaT,18F95F47-1540-4E02-8F1F-CC1BCB6828D0,2014-02-08 10:01:36.827
214,710,"Mountain Bike Socks, L",SO-B909-L,False,True,White,4,3,3.3963,9.50,L,None,None,NaN,0,M,None,U,23.0,18.0,2011-05-31,2012-05-29,NaT,161C035E-21B3-4E14-8E44-AF508F35D80A,2014-02-08 10:01:36.827


In [89]:
# 326 Write a query to retrieve the order IDs along with a new column "OrderPriority" that categorizes orders as "High Priority" if the total due amount is greater than $5000, "Medium Priority" if between $1000 and $5000, and "Low Priority" otherwise

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [90]:
soh['OrderPriority'] = None

In [91]:
soh.loc[soh['TotalDue'] > 5000, 'OrderPriority'] = 'High Priority'

In [92]:
soh.loc[(soh['TotalDue'] > 1000) & (soh['TotalDue'] < 5000), 'OrderPriority'] = 'Meedium Priority'

In [93]:
soh.loc[soh['OrderPriority'].isna(), 'OrderPriority'] = 'Low Priority'

In [94]:
soh[['SalesOrderID', 'OrderPriority']].head(10)

,SalesOrderID,OrderPriority
0,43660,Meedium Priority
1,43661,High Priority
2,43662,High Priority
3,43663,Low Priority
4,43664,High Priority
5,43665,High Priority
6,43666,High Priority
7,43667,High Priority
8,43668,High Priority
9,43669,Low Priority


In [95]:
# 327 Get the list of product names and list prices for products with IDs 4, 8, and 13, and include a count of how many times each product has been ordered

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [96]:
a = p[p['ProductID'].isin([4,8,13])]
a

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827


In [97]:
pro = a[['ProductID', 'Name', 'ListPrice']]
pro

,ProductID,Name,ListPrice
3,4,Headset Ball Bearings,0.0


In [98]:
sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [99]:
sod[sod['ProductID'].isin(pro['ProductID'])]

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate


In [100]:
# 328 Retrieve the employee names and titles for employees with titles similar to 'Engineer', allowing for wildcard matches

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']
emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [101]:
emp[emp['JobTitle'].str.contains('Engineer',case = False)]

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30
5,6,998320692,adventure-works\jossef0,3.0,Design Engineer,1959-03-11,M,M,2008-01-24,True,6,23,True,E39056F1-9CD5-478D-8945-14ACA7FBDCDD,2014-06-30
7,8,811994146,adventure-works\diane1,4.0,Research and Development Engineer,1986-06-05,S,F,2008-12-29,True,62,51,True,31112635-663B-4018-B4A2-A685C0BF48A4,2014-06-30
8,9,658797903,adventure-works\gigi0,4.0,Research and Development Engineer,1979-01-21,M,F,2009-01-16,True,63,51,True,50B6CDC6-7570-47EF-9570-48A64B5F2ECF,2014-06-30
13,14,42487730,adventure-works\michael8,3.0,Senior Design Engineer,1979-06-16,S,M,2010-12-30,True,3,21,True,46286CA4-46DD-4DDB-9128-85B67E98D1A9,2014-06-30
14,15,56920285,adventure-works\sharon0,3.0,Design Engineer,1961-05-02,M,F,2011-01-18,True,4,22,True,54F2FDC0-87C4-4065-A7A8-9AC8EA624235,2014-06-30
290,291,10000001,adventure-works\ken12,NaN,Design Engineer,1952-09-27,S,M,2008-01-24,False,5,23,True,8C5B9C80-0701-4629-9C5E-BD49743A3DF2,2014-06-30


In [102]:
# 329 Retrieve the product names that have no associated product reviews, but only consider products with a list price above the average list price

p =db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [103]:
pr =db_reader().read_table('production.productreview', connection)
pr.head()

,ProductReviewID,ProductID,ReviewerName,ReviewDate,EmailAddress,Rating,Comments,ModifiedDate
0,1,709,John Smith,2013-09-18,john@fourthcoffee.com,5,I can't believe I'm singing the praises of a p...,2013-09-18
1,2,937,David,2013-11-13,david@graphicdesigninstitute.com,4,"A little on the heavy side, but overall the en...",2013-11-13
2,3,937,Jill,2013-11-15,jill@margiestravel.com,4,Maybe it's just because I'm new to mountain bi...,2013-11-15
3,4,798,Laura Norman,2013-11-15,laura@treyresearch.net,5,The Road-550-W from Adventure Works Cycles is ...,2013-11-15


In [104]:
res = p[~p['ProductID'].isin(pr['ProductID'])]
res.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [105]:
p[p['ListPrice'] > p['ListPrice'].mean()].head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.00,1000.00,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
188,516,HL Mountain Seat Assembly,SA-M687,True,False,None,500,375,145.87,196.92,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9E18ADAB-B9C7-45B1-BD95-1805EC4F297D,2014-02-08 10:01:36.827
191,519,HL Road Seat Assembly,SA-R522,True,False,None,500,375,145.87,196.92,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,7B52EE2A-7100-4A39-A0AF-C89012DA6EF8,2014-02-08 10:01:36.827
194,522,HL Touring Seat Assembly,SA-T872,True,False,None,500,375,145.87,196.92,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,8C471BCA-A735-4087-AD50-90EDE0AC1A1B,2014-02-08 10:01:36.827
209,680,"HL Road Frame - Black, 58",FR-R92B-58,True,True,Black,500,375,1059.31,1431.50,58,CM,LB,2.24,1,R,H,U,14.0,6.0,2008-04-30,NaT,NaT,43DD68D6-14A4-461F-9069-55309D90EA7E,2014-02-08 10:01:36.827


In [106]:
# 330 Get the list of orders that do not contain any products with IDs 18, 23, and 28, and also do not contain any products with a list price above $100

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [107]:
sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [108]:
p_d = merge().merge_tbl(p[['ProductID', 'ListPrice']], sod[['SalesOrderID','ProductID']], 'inner', 'ProductID')
p_d.head()

,ProductID,ListPrice,SalesOrderID
0,707,34.99,43665
1,707,34.99,43668
2,707,34.99,43673
3,707,34.99,43677
4,707,34.99,43678


In [109]:
soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [110]:
s = merge().merge_tbl(p_d, soh[['SalesOrderID']], 'inner', 'SalesOrderID')
s.head()

,ProductID,ListPrice,SalesOrderID
0,707,34.99,43665
1,707,34.99,43668
2,707,34.99,43673
3,707,34.99,43677
4,707,34.99,43678


In [111]:
s[s['ProductID'].isin(['18,23,28'])]

,ProductID,ListPrice,SalesOrderID


In [112]:
# 331 Retrieve the product names and list prices for products with a list price less than $50 or a color of 'Red', but exclude products with a weight exceeding 10 pounds

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [113]:
p[(p['ListPrice'] < 50) | (p['Color'] == 'Black') & (p['Weight'] < 10)].head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827
5,317,LL Crankarm,CA-5965,False,False,Black,500,375,0.0,0.0,None,None,None,NaN,0,None,L,None,NaN,NaN,2008-04-30,NaT,NaT,3C9D10B7-A6B2-4774-9963-C19DCEE72FEA,2014-02-08 10:01:36.827


In [114]:
# 332 Get the list of orders placed between '2018-01-01' and '2018-12-31', and include the total number of distinct products ordered in each order

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [115]:
soh[(pd.to_datetime(soh['OrderDate']) > '2018-01-01') & (pd.to_datetime(soh['OrderDate']) < '2018-12-31')].head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate


In [116]:
# 333 Retrieve the customer IDs and names for customers with a total amount spent greater than the average total amount spent by all customers, and also with a total number of orders placed greater than the average number of orders placed by allcustomers

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [117]:
name = db_reader().read_table('person.person', connection)
name.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30


In [118]:
cus = merge().merge_tbl(soh[['SalesOrderID','CustomerID','TotalDue']], name[['BusinessEntityID', 'FirstName', 'LastName']], 'inner', col1 = 'CustomerID', col2 = 'BusinessEntityID')
cus.head()

,SalesOrderID,CustomerID,TotalDue,BusinessEntityID,FirstName,LastName
0,43700,14501,772.5036,14501,Bryan,Gray
1,43701,11003,3756.9890,11003,Jenna,Young
2,43703,16624,3953.9884,16624,Nichole,Goel
3,43704,11005,3729.3640,11005,Joe,Madan
4,43705,11011,3756.9890,11011,Joe,Rana


In [119]:
cus.groupby(['CustomerID', 'FirstName', 'LastName']).agg({'SalesOrderID' : 'count', 'TotalDue' : 'sum'}).rename(columns = {'SalesOrderID' : 'count_f_o', 'TotalDue' : 'total'}).reset_index().query('count_f_o > count_f_o.mean()').head()

,CustomerID,FirstName,LastName,count_f_o,total
0,11000,Mary,Young,3,9115.1341
1,11001,Amber,Young,3,7054.1875
2,11002,Courtney,Young,3,8966.0143
3,11003,Jenna,Young,3,8993.9155
4,11004,Chloe,Harris,3,9056.5911


In [120]:
# 334 Write a query to retrieve the order IDs along with a new column "OrderType" that categorizes orders as "Online" if the order was placed online (using a specific column in the Orders table) and "In-Store" if it was placed in-store

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [121]:
soh['OrderType'] = None

In [122]:
soh.loc[soh['OnlineOrderFlag'] == 1, 'OrderType'] = 'Online'

In [123]:
soh.loc[soh['OnlineOrderFlag'] == 0, 'OrderType'] = 'In-Store'

In [124]:
soh[['SalesOrderID', 'OrderType']].head()

,SalesOrderID,OrderType
0,43660,In-Store
1,43661,In-Store
2,43662,In-Store
3,43663,In-Store
4,43664,In-Store


In [125]:
# 335 Get the list of product names and list prices for products with IDs 4, 8, and 13, and include a count of how many times each product has been ordered, but only consider orders placed in the year 2020

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [126]:
a = p[p['ProductID'].isin([4,8,13])]
a

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827


In [127]:
pro = a[['ProductID', 'Name', 'ListPrice']]
pro

,ProductID,Name,ListPrice
3,4,Headset Ball Bearings,0.0


In [128]:
sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [129]:
sod[sod['ProductID'].isin(pro['ProductID'])]

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate


In [130]:
# 336 Retrieve the employee names and titles for employees with titles containing both the words 'Sales' and 'Representative', in any order

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']
emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [131]:
emp[emp['JobTitle'].str.contains('Sales',case = False) & emp['JobTitle'].str.contains('Representative',case = False)].head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
274,275,841560125,adventure-works\michael9,3.0,Sales Representative,1968-12-25,S,M,2011-05-31,True,38,39,True,E19F8DE2-0F40-418C-84A2-DB48B98DC6A2,2014-06-30
275,276,191644724,adventure-works\linda3,3.0,Sales Representative,1980-02-27,M,F,2011-05-31,True,27,33,True,4509F387-D73A-43DC-A502-B1C27AA1DC9E,2014-06-30
276,277,615389812,adventure-works\jillian0,3.0,Sales Representative,1962-08-29,S,F,2011-05-31,True,24,32,True,A011C97B-081C-4739-996F-C1CAC4532F8D,2014-06-30
277,278,234474252,adventure-works\garrett1,3.0,Sales Representative,1975-02-04,M,M,2011-05-31,True,33,36,True,50EECC16-0D0D-43A9-9649-016C06DE8D78,2014-06-30
278,279,716374314,adventure-works\tsvi0,3.0,Sales Representative,1974-01-18,M,M,2011-05-31,True,29,34,True,BB510FCE-0105-4306-B591-6450D9EBF401,2014-06-30


In [132]:
# 337 Retrieve the product names that have no associated product reviews, but only consider products with a list price above the average list price in their respective category

p =db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [133]:
pr =db_reader().read_table('production.productreview', connection)
pr.head()

,ProductReviewID,ProductID,ReviewerName,ReviewDate,EmailAddress,Rating,Comments,ModifiedDate
0,1,709,John Smith,2013-09-18,john@fourthcoffee.com,5,I can't believe I'm singing the praises of a p...,2013-09-18
1,2,937,David,2013-11-13,david@graphicdesigninstitute.com,4,"A little on the heavy side, but overall the en...",2013-11-13
2,3,937,Jill,2013-11-15,jill@margiestravel.com,4,Maybe it's just because I'm new to mountain bi...,2013-11-15
3,4,798,Laura Norman,2013-11-15,laura@treyresearch.net,5,The Road-550-W from Adventure Works Cycles is ...,2013-11-15


In [134]:
res = p[~p['ProductID'].isin(pr['ProductID'])]
res.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [135]:
psc = db_reader().read_table('production.productsubcategory', connection)
psc.head()

,ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
0,1,1,Mountain Bikes,2D364ADE-264A-433C-B092-4FCBF3804E01,2008-04-30
1,2,1,Road Bikes,000310C0-BCC8-42C4-B0C3-45AE611AF06B,2008-04-30
2,3,1,Touring Bikes,02C5061D-ECDC-4274-B5F1-E91D76BC3F37,2008-04-30
3,4,2,Handlebars,3EF2C725-7135-4C85-9AE6-AE9A3BDD9283,2008-04-30
4,5,2,Bottom Brackets,A9E54089-8A1E-4CF5-8646-E3801F685934,2008-04-30


In [136]:
id = merge().merge_tbl(res[['Name','ProductSubcategoryID', 'ListPrice']], psc[['ProductSubcategoryID', 'ProductCategoryID']], 'inner', 'ProductSubcategoryID')
id.head()

,Name,ProductSubcategoryID,ListPrice,ProductCategoryID
0,"HL Road Frame - Black, 58",14.0,1431.50,2
1,"HL Road Frame - Red, 58",14.0,1431.50,2
2,"Sport-100 Helmet, Red",31.0,34.99,4
3,"Sport-100 Helmet, Black",31.0,34.99,4
4,"Mountain Bike Socks, L",23.0,9.50,3


In [137]:
avg = id.groupby('ProductCategoryID')['ListPrice'].mean().rename('avg').reset_index()
avg

,ProductCategoryID,avg
0,1,146.674000
1,2,472.784135
2,3,50.974286
3,4,34.348966


In [138]:
id = id.merge(avg , on = 'ProductCategoryID')
id.head()

,Name,ProductSubcategoryID,ListPrice,ProductCategoryID,avg
0,"HL Road Frame - Black, 58",14.0,1431.50,2,472.784135
1,"HL Road Frame - Red, 58",14.0,1431.50,2,472.784135
2,"Sport-100 Helmet, Red",31.0,34.99,4,34.348966
3,"Sport-100 Helmet, Black",31.0,34.99,4,34.348966
4,"Mountain Bike Socks, L",23.0,9.50,3,50.974286


In [139]:
id[id['ListPrice'] > id['avg']].head()

,Name,ProductSubcategoryID,ListPrice,ProductCategoryID,avg
0,"HL Road Frame - Black, 58",14.0,1431.50,2,472.784135
1,"HL Road Frame - Red, 58",14.0,1431.50,2,472.784135
2,"Sport-100 Helmet, Red",31.0,34.99,4,34.348966
3,"Sport-100 Helmet, Black",31.0,34.99,4,34.348966
5,"Sport-100 Helmet, Blue",31.0,34.99,4,34.348966


In [140]:
# 338 Get the list of orders that do not contain any products with IDs 18, 23, and 28, and also do not contain any products with a list price above $100, but only consider orders placed by customers located in the 'United States'

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [141]:
sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [142]:
p_d = merge().merge_tbl(p[['ProductID', 'ListPrice']], sod[['SalesOrderID','ProductID']], 'inner', 'ProductID')
p_d.head()

,ProductID,ListPrice,SalesOrderID
0,707,34.99,43665
1,707,34.99,43668
2,707,34.99,43673
3,707,34.99,43677
4,707,34.99,43678


In [143]:
soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [144]:
s = merge().merge_tbl(p_d, soh[['SalesOrderID', 'TerritoryID']], 'inner', 'SalesOrderID')
s.head()

,ProductID,ListPrice,SalesOrderID,TerritoryID
0,707,34.99,43665,1
1,707,34.99,43668,6
2,707,34.99,43673,2
3,707,34.99,43677,6
4,707,34.99,43678,4


In [145]:
a = s[~s['ProductID'].isin(['18,23,28'])]
a.head()

,ProductID,ListPrice,SalesOrderID,TerritoryID
0,707,34.99,43665,1
1,707,34.99,43668,6
2,707,34.99,43673,2
3,707,34.99,43677,6
4,707,34.99,43678,4


In [146]:
lp = a[a['ListPrice'] > 100]
lp.head()

,ProductID,ListPrice,SalesOrderID,TerritoryID
17147,717,1431.5,44283,6
17148,717,1431.5,44288,6
17149,717,1431.5,44314,4
17150,717,1431.5,44520,4
17151,717,1431.5,44531,2


In [147]:
st = db_reader().read_table('sales.salesterritory', connection)
st.head()

,TerritoryID,Name,CountryRegionCode,Group,SalesYTD,SalesLastYear,CostYTD,CostLastYear,rowguid,ModifiedDate
0,1,Northwest,US,North America,7.887187e+06,3.298694e+06,0.0,0.0,43689A10-E30B-497F-B0DE-11DE20267FF7,2008-04-30
1,2,Northeast,US,North America,2.402177e+06,3.607149e+06,0.0,0.0,00FB7309-96CC-49E2-8363-0A1BA72486F2,2008-04-30
2,3,Central,US,North America,3.072175e+06,3.205014e+06,0.0,0.0,DF6E7FD8-1A8D-468C-B103-ED8ADDB452C1,2008-04-30
3,4,Southwest,US,North America,1.051085e+07,5.366576e+06,0.0,0.0,DC3E9EA0-7950-4431-9428-99DBCBC33865,2008-04-30
4,5,Southeast,US,North America,2.538667e+06,3.925071e+06,0.0,0.0,6DC4165A-5E4C-42D2-809D-4344E0AC75E7,2008-04-30


In [148]:
c = db_reader().read_table('person.countryregion', connection)
c.head()

,CountryRegionCode,Name,ModifiedDate
0,AD,Andorra,2008-04-30
1,AE,United Arab Emirates,2008-04-30
2,AF,Afghanistan,2008-04-30
3,AG,Antigua and Barbuda,2008-04-30
4,AI,Anguilla,2008-04-30


In [149]:
cou = merge().merge_tbl(st[['TerritoryID','CountryRegionCode']], c[['CountryRegionCode', 'Name']], 'inner', 'CountryRegionCode')
cou.head()

,TerritoryID,CountryRegionCode,Name
0,1,US,United States
1,2,US,United States
2,3,US,United States
3,4,US,United States
4,5,US,United States


In [150]:
country =  merge().merge_tbl(lp, cou[['TerritoryID', 'Name']], 'inner', 'TerritoryID')
country.head()

,ProductID,ListPrice,SalesOrderID,TerritoryID,Name
0,717,1431.5,44283,6,Canada
1,717,1431.5,44288,6,Canada
2,717,1431.5,44314,4,United States
3,717,1431.5,44520,4,United States
4,717,1431.5,44531,2,United States


In [151]:
country.loc[country['Name'] == 'United States'].head()

,ProductID,ListPrice,SalesOrderID,TerritoryID,Name
2,717,1431.5,44314,4,United States
3,717,1431.5,44520,4,United States
4,717,1431.5,44531,2,United States
5,717,1431.5,44753,4,United States
6,717,1431.5,44763,5,United States


In [152]:
# 339 Retrieve the product names and list prices for products with a list price less than $50 or a color of 'Red', but exclude products with a weight exceeding 10 pounds and which are discontinued

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [153]:
p[((p['ListPrice'] < 50) | (p['Color'] == 'Red')) & (p['Weight'] <= 10) & (p['DiscontinuedDate'].notna())].head()


,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
504,1001,Vintage Handcrafted Wooden Handlebars,VHWH1000,True,True,GOLD,3,4,6.9223,8.9,M,CM,LB,1.5,1,M,H,U,23.0,18.0,2011-05-31 10:00:00,2011-08-31 10:00:00,2012-05-31 10:00:00,4FBD47F2-B7E5-48B0-928E-45B8933A56DA,2014-02-08 10:02:36.827


In [154]:
# 340 Get the list of orders placed between '2018-01-01' and '2018-12-31', and include the total number of distinct products ordered in each order, but only consider orders with a total due amount greater than the average total due amount

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [155]:
soh[(pd.to_datetime(soh['OrderDate']) > '2018-01-01') & (pd.to_datetime(soh['OrderDate']) < '2018-12-31')].head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate


In [156]:
# 341 Retrieve the customer IDs and names for customers with a total amount spent greater than the average total amount spent by all customers, and also with a total number of orders placed greater than the average number of orders placed by allcustomers, but only consider customers who have placed orders in both the 'Online' and 'In-Store' categories

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [157]:
name = db_reader().read_table('person.person', connection)
name.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30


In [158]:
cus = merge().merge_tbl(soh[['SalesOrderID','CustomerID','TotalDue', 'OnlineOrderFlag']], name[['BusinessEntityID', 'FirstName', 'LastName']], 'inner', col1 = 'CustomerID', col2 = 'BusinessEntityID')
cus.head()

,SalesOrderID,CustomerID,TotalDue,OnlineOrderFlag,BusinessEntityID,FirstName,LastName
0,43700,14501,772.5036,True,14501,Bryan,Gray
1,43701,11003,3756.9890,True,11003,Jenna,Young
2,43703,16624,3953.9884,True,16624,Nichole,Goel
3,43704,11005,3729.3640,True,11005,Joe,Madan
4,43705,11011,3756.9890,True,11011,Joe,Rana


In [159]:
a = cus.groupby(['CustomerID', 'FirstName', 'LastName', 'OnlineOrderFlag']).agg({'SalesOrderID' : 'count', 'TotalDue' : 'sum'}).rename(columns = {'SalesOrderID' : 'count_f_o', 'TotalDue' : 'total'}).reset_index().query('count_f_o > count_f_o.mean()')
a.head()

,CustomerID,FirstName,LastName,OnlineOrderFlag,count_f_o,total
0,11000,Mary,Young,True,3,9115.1341
1,11001,Amber,Young,True,3,7054.1875
2,11002,Courtney,Young,True,3,8966.0143
3,11003,Jenna,Young,True,3,8993.9155
4,11004,Chloe,Harris,True,3,9056.5911


In [160]:
a[a['CustomerID'].isin((a['OnlineOrderFlag'] == 1) & (a['OnlineOrderFlag'] == 0))].head()

,CustomerID,FirstName,LastName,OnlineOrderFlag,count_f_o,total


In [161]:
# 342 Write a query to retrieve the order IDs along with a new column "OrderPriority" that categorizes orders as "High Priority" if the total due amount is greater than $5000, "Medium Priority" if between $1000 and $5000, and "Low Priority" otherwise

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [162]:
soh['OrderPriority'] = None

In [163]:
soh.loc[soh['TotalDue'] > 5000, 'OrderPriority'] = 'High Priority'

In [164]:
soh.loc[(soh['TotalDue'] > 1000) & (soh['TotalDue'] < 5000), 'OrderPriority'] = 'Medium Priority'

In [165]:
soh.loc[soh['OrderPriority'].isna(), 'OrderPriority'] = 'Low Priority'

In [166]:
soh[['SalesOrderID', 'OrderPriority']].head(10)

,SalesOrderID,OrderPriority
0,43660,Medium Priority
1,43661,High Priority
2,43662,High Priority
3,43663,Low Priority
4,43664,High Priority
5,43665,High Priority
6,43666,High Priority
7,43667,High Priority
8,43668,High Priority
9,43669,Low Priority


In [167]:
# 343 Get the list of product names and list prices for products with IDs 4, 8, and 13, and include a count of how many times each product has been ordered, but only consider orders placed in the year 2021 and shipped to the 'United States'

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [168]:
a = p[p['ProductID'].isin([4,8,13])]
a

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827


In [169]:
pro = a[['ProductID', 'Name', 'ListPrice']]
pro

,ProductID,Name,ListPrice
3,4,Headset Ball Bearings,0.0


In [170]:
sod = db_reader().read_table('sales.salesorderdetail', connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [171]:
sod[sod['ProductID'].isin(pro['ProductID'])]

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate


In [172]:
# 344 Retrieve the employee names and titles for employees with titles containing both the words 'Sales' and 'Manager', in any order, and who have been hired after '2010-01-01'

col = ['BusinessEntityID',
    'NationalIDNumber',
    'LoginID',
    'OrganizationLevel',
    'JobTitle',
    'BirthDate',
    'MaritalStatus',
    'Gender',
    'HireDate',
    'SalariedFlag',
    'VacationHours',
    'SickLeaveHours',
    'CurrentFlag',
    'rowguid',
    'ModifiedDate']
emp = db_reader().read_table('humanresources.employee', connection, col)
emp.head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
0,1,295847284,adventure-works\ken0,NaN,Chief Executive Officer,1969-01-29,S,M,2009-01-14,True,99,69,True,F01251E5-96A3-448D-981E-0F99D789110D,2014-06-30
1,2,245797967,adventure-works\terri0,1.0,Vice President of Engineering,1971-08-01,S,F,2008-01-31,True,1,20,True,45E8F437-670D-4409-93CB-F9424A40D6EE,2014-06-30
2,3,509647174,adventure-works\roberto0,2.0,Engineering Manager,1974-11-12,M,M,2007-11-11,True,2,21,True,9BBBFB2C-EFBB-4217-9AB7-F97689328841,2014-06-30
3,4,112457891,adventure-works\rob0,3.0,Senior Tool Designer,1974-12-23,S,M,2007-12-05,False,48,80,True,59747955-87B8-443F-8ED4-F8AD3AFDF3A9,2014-06-30
4,5,695256908,adventure-works\gail0,3.0,Design Engineer,1952-09-27,M,F,2008-01-06,True,5,22,True,EC84AE09-F9B8-4A15-B4A9-6CCBAB919B08,2014-06-30


In [173]:
emp[emp['JobTitle'].str.contains('Sales',case = False) & emp['JobTitle'].str.contains('Manager',case = False) & (pd.to_datetime(emp['HireDate']) > '2010-01-01')].head()

,BusinessEntityID,NationalIDNumber,LoginID,OrganizationLevel,JobTitle,BirthDate,MaritalStatus,Gender,HireDate,SalariedFlag,VacationHours,SickLeaveHours,CurrentFlag,rowguid,ModifiedDate
273,274,502097814,adventure-works\stephen0,2.0,North American Sales Manager,1951-10-17,M,M,2011-01-04,True,14,27,True,86D5FA9E-4BC0-48C8-91DC-1EC467418D11,2014-06-30
284,285,481044938,adventure-works\syed0,2.0,Pacific Sales Manager,1975-01-11,M,M,2013-03-14,True,20,30,True,86F292DB-B73C-429D-9912-800994D809FB,2014-06-30
286,287,982310417,adventure-works\amy0,2.0,European Sales Manager,1957-09-20,M,F,2012-04-16,True,21,30,True,66D66445-EE78-4676-9E66-0E22D6109A92,2014-06-30


In [174]:
# 345 Retrieve the product names that have no associated product reviews, but only consider products with a list price above the average list price in their respective category and which are not discontinued

p =db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [175]:
pr =db_reader().read_table('production.productreview', connection)
pr.head()

,ProductReviewID,ProductID,ReviewerName,ReviewDate,EmailAddress,Rating,Comments,ModifiedDate
0,1,709,John Smith,2013-09-18,john@fourthcoffee.com,5,I can't believe I'm singing the praises of a p...,2013-09-18
1,2,937,David,2013-11-13,david@graphicdesigninstitute.com,4,"A little on the heavy side, but overall the en...",2013-11-13
2,3,937,Jill,2013-11-15,jill@margiestravel.com,4,Maybe it's just because I'm new to mountain bi...,2013-11-15
3,4,798,Laura Norman,2013-11-15,laura@treyresearch.net,5,The Road-550-W from Adventure Works Cycles is ...,2013-11-15


In [176]:
res = p[~p['ProductID'].isin(pr['ProductID'])]
res.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [177]:
a = p[p['ListPrice'] > p['ListPrice'].mean()]
a.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.00,1000.00,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
188,516,HL Mountain Seat Assembly,SA-M687,True,False,None,500,375,145.87,196.92,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9E18ADAB-B9C7-45B1-BD95-1805EC4F297D,2014-02-08 10:01:36.827
191,519,HL Road Seat Assembly,SA-R522,True,False,None,500,375,145.87,196.92,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,7B52EE2A-7100-4A39-A0AF-C89012DA6EF8,2014-02-08 10:01:36.827
194,522,HL Touring Seat Assembly,SA-T872,True,False,None,500,375,145.87,196.92,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,8C471BCA-A735-4087-AD50-90EDE0AC1A1B,2014-02-08 10:01:36.827
209,680,"HL Road Frame - Black, 58",FR-R92B-58,True,True,Black,500,375,1059.31,1431.50,58,CM,LB,2.24,1,R,H,U,14.0,6.0,2008-04-30,NaT,NaT,43DD68D6-14A4-461F-9069-55309D90EA7E,2014-02-08 10:01:36.827


In [178]:
a[a['DiscontinuedDate'].isna()].head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.00,1000.00,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
188,516,HL Mountain Seat Assembly,SA-M687,True,False,None,500,375,145.87,196.92,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9E18ADAB-B9C7-45B1-BD95-1805EC4F297D,2014-02-08 10:01:36.827
191,519,HL Road Seat Assembly,SA-R522,True,False,None,500,375,145.87,196.92,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,7B52EE2A-7100-4A39-A0AF-C89012DA6EF8,2014-02-08 10:01:36.827
194,522,HL Touring Seat Assembly,SA-T872,True,False,None,500,375,145.87,196.92,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,8C471BCA-A735-4087-AD50-90EDE0AC1A1B,2014-02-08 10:01:36.827
209,680,"HL Road Frame - Black, 58",FR-R92B-58,True,True,Black,500,375,1059.31,1431.50,58,CM,LB,2.24,1,R,H,U,14.0,6.0,2008-04-30,NaT,NaT,43DD68D6-14A4-461F-9069-55309D90EA7E,2014-02-08 10:01:36.827


In [179]:
# 346 Get the list of orders that do not contain any products with IDs 18, 23, and 28, and also do not contain any products with a list price above $100, but only consider orders placed by customers located in the 'United States' and shipped to the 'Canada'

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [180]:
p_d = merge().merge_tbl(p[['ProductID', 'ListPrice']], sod[['SalesOrderID','ProductID']], 'inner', 'ProductID')
p_d.head()

,ProductID,ListPrice,SalesOrderID
0,707,34.99,43665
1,707,34.99,43668
2,707,34.99,43673
3,707,34.99,43677
4,707,34.99,43678


In [181]:
soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [182]:
s = merge().merge_tbl(p_d, soh[['SalesOrderID', 'TerritoryID' ,'ShipToAddressID']], 'inner', 'SalesOrderID')
s.head()

,ProductID,ListPrice,SalesOrderID,TerritoryID,ShipToAddressID
0,707,34.99,43665,1,849
1,707,34.99,43668,6,529
2,707,34.99,43673,2,821
3,707,34.99,43677,6,556
4,707,34.99,43678,4,1021


In [183]:
a = s[~s['ProductID'].isin(['18,23,28'])]
a.head()

,ProductID,ListPrice,SalesOrderID,TerritoryID,ShipToAddressID
0,707,34.99,43665,1,849
1,707,34.99,43668,6,529
2,707,34.99,43673,2,821
3,707,34.99,43677,6,556
4,707,34.99,43678,4,1021


In [184]:
lp = a[a['ListPrice'] > 100]
lp.head()

,ProductID,ListPrice,SalesOrderID,TerritoryID,ShipToAddressID
17147,717,1431.5,44283,6,482
17148,717,1431.5,44288,6,529
17149,717,1431.5,44314,4,578
17150,717,1431.5,44520,4,1001
17151,717,1431.5,44531,2,829


In [185]:
st = db_reader().read_table('sales.salesterritory', connection)
st.head()

,TerritoryID,Name,CountryRegionCode,Group,SalesYTD,SalesLastYear,CostYTD,CostLastYear,rowguid,ModifiedDate
0,1,Northwest,US,North America,7.887187e+06,3.298694e+06,0.0,0.0,43689A10-E30B-497F-B0DE-11DE20267FF7,2008-04-30
1,2,Northeast,US,North America,2.402177e+06,3.607149e+06,0.0,0.0,00FB7309-96CC-49E2-8363-0A1BA72486F2,2008-04-30
2,3,Central,US,North America,3.072175e+06,3.205014e+06,0.0,0.0,DF6E7FD8-1A8D-468C-B103-ED8ADDB452C1,2008-04-30
3,4,Southwest,US,North America,1.051085e+07,5.366576e+06,0.0,0.0,DC3E9EA0-7950-4431-9428-99DBCBC33865,2008-04-30
4,5,Southeast,US,North America,2.538667e+06,3.925071e+06,0.0,0.0,6DC4165A-5E4C-42D2-809D-4344E0AC75E7,2008-04-30


In [186]:
c = db_reader().read_table('person.countryregion', connection)
c.head()

,CountryRegionCode,Name,ModifiedDate
0,AD,Andorra,2008-04-30
1,AE,United Arab Emirates,2008-04-30
2,AF,Afghanistan,2008-04-30
3,AG,Antigua and Barbuda,2008-04-30
4,AI,Anguilla,2008-04-30


In [187]:
cou = merge().merge_tbl(st[['TerritoryID','CountryRegionCode']], c[['CountryRegionCode', 'Name']], 'inner', 'CountryRegionCode')
cou.head()

,TerritoryID,CountryRegionCode,Name
0,1,US,United States
1,2,US,United States
2,3,US,United States
3,4,US,United States
4,5,US,United States


In [188]:
country =  merge().merge_tbl(lp, cou[['TerritoryID', 'Name']], 'inner', 'TerritoryID')
country.head()

,ProductID,ListPrice,SalesOrderID,TerritoryID,ShipToAddressID,Name
0,717,1431.5,44283,6,482,Canada
1,717,1431.5,44288,6,529,Canada
2,717,1431.5,44314,4,578,United States
3,717,1431.5,44520,4,1001,United States
4,717,1431.5,44531,2,829,United States


In [189]:
ans = country.loc[country['Name'] == 'United States']
ans.head()

,ProductID,ListPrice,SalesOrderID,TerritoryID,ShipToAddressID,Name
2,717,1431.5,44314,4,578,United States
3,717,1431.5,44520,4,1001,United States
4,717,1431.5,44531,2,829,United States
5,717,1431.5,44753,4,1004,United States
6,717,1431.5,44763,5,984,United States


In [190]:
col = [
    "AddressID", 
    "AddressLine1", 
    "AddressLine2", 
    "City", 
    "StateProvinceID", 
    "PostalCode", 
    "rowguid", 
    "ModifiedDate"
]

add = db_reader().read_table('person.address', connection, col)
add.head()

,AddressID,AddressLine1,AddressLine2,City,StateProvinceID,PostalCode,rowguid,ModifiedDate
0,1,1970 Napa Ct.,None,Bothell,79,98011,9AADCB0D-36CF-483F-84D8-585C2D4EC6E9,2007-12-04
1,2,9833 Mt. Dias Blv.,None,Bothell,79,98011,32A54B9E-E034-4BFB-B573-A71CDE60D8C0,2008-11-30
2,3,7484 Roundtree Drive,None,Bothell,79,98011,4C506923-6D1B-452C-A07C-BAA6F5B142A4,2013-03-07
3,4,9539 Glenside Dr,None,Bothell,79,98011,E5946C78-4BCC-477F-9FA1-CC09DE16A880,2009-02-03
4,5,1226 Shoe St.,None,Bothell,79,98011,FBAFF937-4A97-4AF0-81FD-B849900E9BB0,2008-12-19


In [191]:
s = db_reader().read_table('person.[StateProvince]', connection)
s.head()

,StateProvinceID,StateProvinceCode,CountryRegionCode,IsOnlyStateProvinceFlag,Name,TerritoryID,rowguid,ModifiedDate
0,1,AB,CA,False,Alberta,6,298C2880-AB1C-4982-A5AD-A36EB4BA0D34,2014-02-08 10:17:21.587
1,2,AK,US,False,Alaska,1,5B7B8462-A888-4E0B-A3E1-7278F8AF107E,2014-02-08 10:17:21.587
2,3,AL,US,False,Alabama,5,41B328BE-21AE-45D0-841D-6F8DD71CE626,2014-02-08 10:17:21.587
3,4,AR,US,False,Arkansas,3,54656A80-06F2-4C70-BA10-247179FC246E,2014-02-08 10:17:21.587
4,5,AS,AS,True,American Samoa,1,255D15E1-9F6E-4CF8-9E5F-6B3858AD9B6A,2014-02-08 10:17:21.587


In [192]:
state = merge().merge_tbl(add[['AddressID','StateProvinceID']], s[['StateProvinceID', 'CountryRegionCode']], 'inner', 'StateProvinceID')
state.head()

,AddressID,StateProvinceID,CountryRegionCode
0,1,79,US
1,2,79,US
2,3,79,US
3,4,79,US
4,5,79,US


In [193]:
ct = merge().merge_tbl(state[['AddressID','CountryRegionCode']], cou[['CountryRegionCode', 'Name']], 'inner', 'CountryRegionCode')
ct.head()

,AddressID,CountryRegionCode,Name
0,1,US,United States
1,1,US,United States
2,1,US,United States
3,1,US,United States
4,1,US,United States


In [194]:
result = merge().merge_tbl(ans, ct[['AddressID', 'Name']], 'inner', col1 = 'ShipToAddressID', col2 = 'AddressID')
result.head()

,ProductID,ListPrice,SalesOrderID,TerritoryID,ShipToAddressID,Name_x,AddressID,Name_y
0,717,1431.5,44314,4,578,United States,578,United States
1,717,1431.5,44314,4,578,United States,578,United States
2,717,1431.5,44314,4,578,United States,578,United States
3,717,1431.5,44314,4,578,United States,578,United States
4,717,1431.5,44314,4,578,United States,578,United States


In [195]:
result[result['Name_y'] == 'Canada'].head()

,ProductID,ListPrice,SalesOrderID,TerritoryID,ShipToAddressID,Name_x,AddressID,Name_y


In [196]:
# 347 Retrieve the product names and list prices for products with a list price less than $50 or a color of 'Red', but exclude products with a weight exceeding 10 pounds and which are discontinued, and have been ordered at least 10 times

p = db_reader().read_table('production.product', connection)
p.head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
0,1,Adjustable Race,AR-5381,False,False,None,1000,750,0.0,1000.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,694215B7-08F7-4C0D-ACB1-D734BA44C0C8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,False,False,None,1000,750,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,58AE3C20-4F3A-4749-A7D4-D568806CC537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,9C21AED2-5BFA-4F18-BCB8-F11638DC2E4E,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,False,False,None,800,600,0.0,0.0,None,None,None,NaN,0,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,ECFED6CB-51FF-49B5-B06C-7D8AC834DB8B,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,True,False,None,800,600,0.0,0.0,None,None,None,NaN,1,None,None,None,NaN,NaN,2008-04-30,NaT,NaT,E73E9750-603B-4131-89F5-3DD15ED5FF80,2014-02-08 10:01:36.827


In [197]:
a = p[((p['ListPrice'] < 50) | (p['Color'] == 'Red')) & (p['Weight'] <= 10) & (p['DiscontinuedDate'].notna())].head()

In [198]:
sod = db_reader().read_table('sales.salesorderdetail',connection)
sod.head()

,SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
0,43660,13,6431-4D57-83,1,762,1,419.4589,0.0,419.4589,419A1302-AC7A-4044-97B2-66D9D14CD02E,2011-05-31
1,43660,14,6431-4D57-83,1,758,1,874.7940,0.0,874.7940,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-31
2,43661,15,4E0A-4F89-AE,1,745,1,809.7600,0.0,809.7600,EDE1759E-6733-4C7B-A43F-DC6F48002D8A,2011-05-31
3,43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-31
4,43661,17,4E0A-4F89-AE,2,747,1,714.7043,0.0,1429.4086,B136852E-24C9-4006-8048-B14AEFE6C337,2011-05-31


In [199]:
a[a['ProductID'].isin(sod['ProductID'])].head()

,ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate


In [200]:
# 348 Get the list of orders placed between '2019-01-01' and '2019-12-31', and include the total number of distinct products ordered in each order, but only consider orders with a total due amount greater than the average total due amount and with a payment method of 'Credit Card'

soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [201]:
soh[(pd.to_datetime(soh['OrderDate']) > '2019-01-01') & (pd.to_datetime(soh['OrderDate']) < '2019-12-31')].head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate


In [202]:
# 349 Retrieve the customer IDs and names for customers with a total amount spent greater than the average total amount 
#spent by all customers, and also with a total number of orders placed greater than the average number of orders placed by all
#customers, but only consider customers who have placed orders in both the 'Online' and 'In-Store' categories, and have not 
#placed any orders in the year 2022

In [203]:
soh = db_reader().read_table('sales.salesorderheader', connection)
soh.head()

,SalesOrderID,RevisionNumber,OrderDate,DueDate,ShipDate,Status,OnlineOrderFlag,SalesOrderNumber,PurchaseOrderNumber,AccountNumber,CustomerID,SalesPersonID,TerritoryID,BillToAddressID,ShipToAddressID,ShipMethodID,CreditCardID,CreditCardApprovalCode,CurrencyRateID,SubTotal,TaxAmt,Freight,TotalDue,Comment,rowguid,ModifiedDate
0,43660,9,2011-05-20,2011-06-12,2011-06-07,5,False,SO43660,PO18850127500,10-4020-000117,29672,279.0,5,921,921,5,5618.0,115213Vi29411,NaN,1294.2529,124.2483,38.8276,1457.3288,None,738DC42D-D03B-48A1-9822-F95A67EA7389,2011-06-07
1,43661,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43661,PO18473189620,10-4020-000442,29734,282.0,6,517,517,5,1346.0,85274Vi6854,4.0,32726.4786,3153.7696,985.5530,36865.8012,None,D91B9131-18A4-4A11-BC3A-90B6F53E9D74,2011-06-07
2,43662,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43662,PO18444174044,10-4020-000227,29994,282.0,6,482,482,5,10456.0,125295Vi53935,4.0,28832.5289,2775.1646,867.2389,32474.9324,None,4A1ECFC0-CC3A-4740-B028-1C50BB48711C,2011-06-07
3,43663,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43663,PO18009186470,10-4020-000510,29565,276.0,4,1073,1073,5,4322.0,45303Vi22691,NaN,419.4589,40.2681,12.5838,472.3108,None,9B1E7A40-6AE0-4AD3-811C-A64951857C4B,2011-06-07
4,43664,8,2011-05-31,2011-06-12,2011-06-07,5,False,SO43664,PO16617121983,10-4020-000397,29898,280.0,1,876,876,5,806.0,95555Vi4081,NaN,24432.6088,2344.9921,732.8100,27510.4109,None,22A8A5DA-8C22-42AD-9241-839489B6EF0D,2011-06-07


In [204]:
name = db_reader().read_table('person.person', connection)
name.head()

,BusinessEntityID,PersonType,NameStyle,Title,FirstName,MiddleName,LastName,Suffix,EmailPromotion,AdditionalContactInfo,Demographics,rowguid,ModifiedDate
0,1,EM,False,None,Ken,J,Sánchez,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",92C4279F-1207-48A3-8448-4636514EB7E2,2009-01-07
1,2,EM,False,None,Terri,Lee,Duffy,None,1,None,"<IndividualSurvey xmlns=""http://schemas.micros...",D8763459-8AA8-47CC-AFF7-C9079AF79033,2008-01-24
2,3,EM,False,None,Roberto,None,Tamburello,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",E1A2555E-0828-434B-A33B-6F38136A37DE,2007-11-04
3,4,EM,False,None,Rob,None,Walters,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F2D7CE06-38B3-4357-805B-F4B6B71C01FF,2007-11-28
4,5,EM,False,Ms.,Gail,A,Erickson,None,0,None,"<IndividualSurvey xmlns=""http://schemas.micros...",F3A3F6B4-AE3B-430C-A754-9F2231BA6FEF,2007-12-30


In [205]:
cus = merge().merge_tbl(soh[['SalesOrderID','CustomerID','TotalDue', 'OrderDate','OnlineOrderFlag']], name[['BusinessEntityID', 'FirstName', 'LastName']], 'inner', col1 = 'CustomerID', col2 = 'BusinessEntityID')
cus.head()

,SalesOrderID,CustomerID,TotalDue,OrderDate,OnlineOrderFlag,BusinessEntityID,FirstName,LastName
0,43700,14501,772.5036,2011-05-31,True,14501,Bryan,Gray
1,43701,11003,3756.9890,2011-05-31,True,11003,Jenna,Young
2,43703,16624,3953.9884,2011-06-01,True,16624,Nichole,Goel
3,43704,11005,3729.3640,2011-06-01,True,11005,Joe,Madan
4,43705,11011,3756.9890,2011-06-01,True,11011,Joe,Rana


In [206]:
a = cus.groupby(['CustomerID', 'FirstName', 'LastName', 'OrderDate','OnlineOrderFlag']).agg({'SalesOrderID' : 'count', 'TotalDue' : 'sum'}).rename(columns = {'SalesOrderID' : 'count_f_o', 'TotalDue' : 'total'}).reset_index().query('count_f_o > count_f_o.mean()')
a.head()

,CustomerID,FirstName,LastName,OrderDate,OnlineOrderFlag,count_f_o,total
222,11078,Joe,Sanz,2014-03-15,True,2,66.8194
452,11153,Marshall,He,2013-08-29,True,2,121.2959
495,11176,Morgan,Miller,2013-07-13,True,2,114.6438
677,11212,Marshall,Cai,2013-10-03,True,2,57.7695
681,11212,Marshall,Cai,2014-02-10,True,2,209.9169


In [207]:
a[a['CustomerID'].isin((a['OnlineOrderFlag'] == 1) & (a['OnlineOrderFlag'] == 0)) & (pd.to_datetime(a['OrderDate'].dt.year) == 2022)].head()

,CustomerID,FirstName,LastName,OrderDate,OnlineOrderFlag,count_f_o,total
